<code>pip install keras-octave-conv</code>
    
Use [keras-octave-conv](https://pypi.org/project/keras-octave-conv/) before official implementation

In [1]:
import os
import tensorflow as tf

#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

GPU found


In [8]:
from keras.layers import Input, BatchNormalization, MaxPool2D, Flatten, Dropout, Dense
from keras.models import Model
from keras_octave_conv import OctaveConv2D

inputs = Input(shape=(28, 28, 1))
high, low = OctaveConv2D(filters=32, kernel_size=3)(inputs)

high, low = MaxPool2D()(high), MaxPool2D()(low)
high, low = OctaveConv2D(filters=16, kernel_size=3)([high, low])

conv = OctaveConv2D(filters=8, kernel_size=3, ratio_out=0.0)([high, low])
pool = MaxPool2D()(conv)

flatten = Flatten()(pool)
dense = Dense(units=64, activation='relu')(flatten)
outputs = Dense(units=10, activation='softmax')(dense)

model = Model(inputs=inputs, outputs=outputs)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
octave_conv2d_4 (OctaveConv2D)  [(None, 28, 28, 16), 320         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 14, 14, 16)   0           octave_conv2d_4[0][0]            
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 7, 7, 16)     0           octave_conv2d_4[0][1]            
__________________________________________________________________________________________________
octave_con

In [9]:
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print(train_images.shape)
print(test_images.shape)

(60000, 28, 28)
(10000, 28, 28)


In [10]:
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

In [11]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [12]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 9s 157us/step - loss: 0.2011 - acc: 0.9397
Epoch 2/5
60000/60000 [==============================] - 9s 150us/step - loss: 0.0698 - acc: 0.9786 1s 
Epoch 3/5
60000/60000 [==============================] - 9s 156us/step - loss: 0.0506 - acc: 0.9844
Epoch 4/5
60000/60000 [==============================] - 9s 142us/step - loss: 0.0393 - acc: 0.9876
Epoch 5/5
60000/60000 [==============================] - 9s 146us/step - loss: 0.0331 - acc: 0.9901


In [14]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

10000/10000 [==============================] - 1s 103us/step
0.9864
